In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
import string

spark = SparkSession \
            .builder \
            .appName("StructuredNetworkWordCount") \
            .getOrCreate()


In [6]:
static = spark.read.json("/home/jovyan/work/data/flight-data/json")
dataSchema = static.schema
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
  .json("/home/jovyan/work/data/flight-data/json")
streaming.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [7]:
counts = streaming.groupBy("DEST_COUNTRY_NAME").count()

In [8]:
query = counts.writeStream.queryName("counts").format("memory").outputMode("complete").start()

In [10]:
from time import sleep
for x in range(5):
    spark.sql("SELECT * FROM counts").show()
    sleep(1)

+-----------------+-----+
|DEST_COUNTRY_NAME|count|
+-----------------+-----+
|             Chad|    1|
|         Paraguay|    6|
|         Anguilla|    6|
|           Russia|    6|
|            Yemen|    1|
|          Senegal|    6|
|           Sweden|    6|
|         Kiribati|    6|
|           Guyana|    6|
|      Philippines|    6|
|         Djibouti|    1|
|         Malaysia|    5|
|        Singapore|    5|
|             Fiji|    6|
|           Turkey|    6|
|           Malawi|    1|
|             Iraq|    1|
|          Germany|    6|
|      Afghanistan|    3|
|           Rwanda|    1|
+-----------------+-----+
only showing top 20 rows

+-----------------+-----+
|DEST_COUNTRY_NAME|count|
+-----------------+-----+
|             Chad|    1|
|         Paraguay|    6|
|         Anguilla|    6|
|           Russia|    6|
|            Yemen|    1|
|          Senegal|    6|
|           Sweden|    6|
|         Kiribati|    6|
|           Guyana|    6|
|      Philippines|    6|
|         Dj